# Resim Üzerinde Maske Tanımlama
Eğitilen **Maske Tanıma Modeli**, gerçek uygulamalarda kullanılabilecek olgunluktadır. Bu amaçla, herhangi bir resim içindeki kişilerin maske takıp takmadıklarının belirlenmesi için üç benzer uygulama geliştirilmiştir. 
Geliştirilen maske tanımlama modelini kullanabilmek için, öncelikle resimlerdeki yüz nesnelerinin belirlenmesi gerekmektedir. Resimlerdeki yüz belirleme algoritması, bu ödevin konusu olmadığı için, bu amaçla kullanımı genel kabul görmüş üç farklı algoritma seçilmiş ve maske tanımlaması, seçilmiş olan üç farklı yüz belirleme algoritmasının çıktıları üzerinde denenmiştir. Bu amaçla seçilen üç farklı Python algoritma kütüphanesi şunlardır:
* OpenCV Haar
* OpenCV DNN
* MTCNN



In [1]:
import argparse

import cv2
import matplotlib.pyplot as plt
import numpy as np

from mtcnn.mtcnn import MTCNN

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array


# OpenCV Haar
OpenCV, yüz algılamada **Haar Kademeleri**ni kullanmaktadır. Haar kademelerini kullanarak yüz algılama, kademeli işlevin bir dizi giriş verisiyle eğitildiği makine öğrenimi tabanlı bir yaklaşımdır. OpenCV halihazırda yüz, gözler, gülümsemeler vb. İçin önceden eğitilmiş birçok sınıflandırıcı içermektedir.

OpenCv’nin Haar sınıflandırıcısını kullanabilmek için, OpenCv’nin GitHub deposunda bulunan eğitimli sınıflandırıcı XML dosyasının (haarcascade_frontalface_default.xml) yazılım ortamına indirilmesi gerekmektedir. Bu dosya, <haarcascade_frontalface_default.xml> olarak proje OpenCV GitHub Repository’sindeki <face_detector> klasöründe de mevcuttur.
(https://github.com/opencv/opencv/tree/master/data/haarcascades)

# OpenCV Haar Sınıflandırıcısı Kullanan Yüz Maskesi Tanımlama Fonksiyonu

In [5]:
def FaceMaskDetection_Picture_With_OpenCV_HaarCascade(image_file, model_file, cascade_classifier):
    faceCascade = cv2.CascadeClassifier(cascade_classifier)

    model = load_model(model_file)

    image = plt.imread(image_file)

    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    print("> {0} adet yüz bulundu!".format(len(faces)))

    # Bulunan yüzleri dörgen ile çerçeveleyelim
    for (x, y, w, h) in faces:
        startX = x
        startY = y
        endX = x + w
        endY = y + h

        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # Elde edilen yüzü, maske varlığını test etmek için modelimize yolluyoruz
        (withoutMask, mask) = model.predict(face)[0]

        # Modelimizin değerlendirmesi sonucunda elde ettiğimiz sınıf etiketlerini
        #   kullanarak, sonuç ve sonuç karesinin rengini belirliyoruz
        label = "Maske Var" if mask > withoutMask else "Maske Yok"
        color = (0, 255, 0) if label == "Maske Var" else (0, 0, 255)

        # Oluşturulan etikete modelin döndürdüğü olasılığı da ekliyoruz
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # Çıktı çerçevesine etiketi ve yüzü çevreleyen dörtgeni çiziyoruz
        cv2.putText(image, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
        cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)

    # İşlenen görüntüyü sonuç olarak ekrana ve diske yazdırıyoruz
    cv2.putText(image, "[With OpenCV CascadeClassifier] *** NOT OK ***", (5,20), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (255, 0, 0), 1)
    cv2.imwrite("DetectMask_OpenCV_HaarCascade.jpg", image, )
    cv2.imshow("Output", image)
    cv2.waitKey(0)


# OpenCV DNN
OpenCV Deep Neural Networks (**OpenCV DNN**), Single Shot-Multibox Detector (SSD) tabanlı bir Caffe modelidir ve omurgası olarak ResNet-10 mimarisini kullanır. OpenCV 3.3 sonrası derin sinir ağı modülünde tanıtılmıştır.

OpenCV DNN’i kullanabilmek için gereken eğitimli DNN dosyası <deploy.prototxt> ve hesaplanmış hazır ağırlıkları içeren <res10_300x300_ssd_iter_140000.caffemodel> dosyasını OpenCv’nin GitHub deposundan yazılım geliştirme ortamına indirilmelidir.
(https://github.com/opencv/opencv/tree/master/samples/dnn)


# OpenCV DNN Sınıflandırıcısı Kullanan Yüz Maskesi Tanımlama Fonksiyonu

In [17]:
def FaceMaskDetection_Picture_With_OpenCV_DNN(image_file, model_file, prototxt_file, weights_file, face_confidence_probability):
	#
	print("> DNN yüz tanımlama modeli yükleniyor...")
	# OpenCV DNN modelin diskten okuyalım
	net = cv2.dnn.readNet(prototxt_file, weights_file)

	# Maske belirleme amacıyla oluşturduğumuz modeli diskten okuyalım
	print("> Yüz maskesi tanımlama modeli yükleniyor...")
	model = load_model(model_file)

	image = cv2.imread(image_file)

	(h, w) = image.shape[:2]

	# Yüklenen görüntüden blop oluşturalım
	blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# Oluşturulan blob'u var olan potansiyel yüzleri algılamak için DNN'e yollayalım
	print("> Yüzler algılanıyor...")
	net.setInput(blob)
	detections = net.forward()

	print("> {0} adet yüz olmaya aday nesne bulundu!".format(detections.shape[2]))

	faceCount = 0
	# Bulunan bütün yüzler için
	for i in range(0, detections.shape[2]):
		# Geri yollanan nesnenin yüz olma olasılığını alalım
		confidence = detections[0, 0, i, 2]

		# Yüzdesi parametre olarak yolladığımız değerden büyük olan yüzleri inceleyelim
		if confidence > face_confidence_probability:
			# Yüz nesnesinin etrafına çizeceğimiz diktörgen için ilgili koordinatlarını belirleyelim
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# Belirlediğimiz koordinatlar dışa taşmasın
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# Yüzü oluşturduğumuz modele yollamak için hazırlık yapalım
			face = image[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)
			face = np.expand_dims(face, axis=0)

			# Elde edilen final görüntüyü işlemek için oluşturduğumuz modelimize yollayalım
			(withoutMask, mask) = model.predict(face)[0]

			# Modelimizin değerlendirmesi sonucunda elde ettiğimiz sınıf etiketlerini
			# 	kullanarak, sonuç ve sonuç karesinin rengini belirliyoruz
			label = "Maske Var" if mask > withoutMask else "Maske Yok"
			color = (0, 255, 0) if label == "Maske Var" else (0, 0, 255)

			# Oluşturulan etikete modelin döndürdüğü olasılığı da ekliyoruz
			label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

			# Çıktı çerçevesine etiketi ve yüzü çevreleyen dörtgeni çiziyoruz
			cv2.putText(image, label, (startX, startY - 10),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
			cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
			#
			faceCount=faceCount+1

	print("> {0} adet yüz belirlendi!".format(faceCount))
	# İşlenen görüntüyü sonuç olarak ekrana ve diske yazdırıyoruz
	cv2.putText(image, "[With OpenCV DNN] *** NOT OK ***", (5,20), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (255, 0, 0), 1)
	cv2.imwrite("DetectMask_OpenCV_DNN.jpg", image, )
	cv2.imshow("Output", image)
	cv2.waitKey(0)


# MTCNN (Multi-task Cascaded Convolutional Neural Networks)
MTCNN (**Multi-task Cascaded Convolutional Neural Networks**), bir görüntüdeki yüzlerin sınırlayıcı kutularını 5 Noktalı Yüz İşaretleri ile algılayan 3 aşamadan oluşan bir algoritmadır. Her aşama, girişlerini bir CNN (**Convolutional Neural Network**)'den geçirerek algılama sonuçlarını kademeli olarak iyileştirerek sınırlayıcı kutu adaylarını puanlarıyla birlikte döndürür. Değerlendirilen üç algoritmadan, belirli bir resimde en iyi sonucu bu algoritma vermektedir.

# MTCNN Sınıflandırıcısı Kullanan Yüz Maskesi Tanımlama Fonksiyonu

In [19]:
def FaceMaskDetection_Picture_With_MTCNN(image_file, model_file):
    detector=MTCNN()

    model = load_model(model_file)

    image = plt.imread(image_file)

    faces=detector.detect_faces(image)

    print("> {0} adet yüz bulundu!".format(len(faces)))

    for face in faces:
        bounding_box = face['box']

        startX = bounding_box[0]
        startY = bounding_box[1]
        endX = bounding_box[0]+bounding_box[2]
        endY = bounding_box[1] + bounding_box[3]

        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # Elde edilen yüzü, maske varlığını test etmek için modelimize yolluyoruz
        (withoutMask, mask) = model.predict(face)[0]

        # Modelimizin değerlendirmesi sonucunda elde ettiğimiz sınıf etiketlerini
        #   kullanarak, sonuç ve sonuç karesinin rengini belirliyoruz
        label = "Maske Var" if mask > withoutMask else "Maske Yok"
        color = (0, 255, 0) if label == "Maske Var" else (0, 0, 255)

        # Oluşturulan etikete modelin döndürdüğü olasılığı da ekliyoruz
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # Çıktı çerçevesine etiketi ve yüzü çevreleyen dörtgeni çiziyoruz
        cv2.putText(image, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

    # İşlenen görüntüyü sonuç olarak ekrana ve diske yazdırıyoruz
    cv2.putText(image, "[With MTCNN] *** OK ***", (5,20), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (255, 0, 0), 1)
    cv2.imwrite("DetectMask_MTCNN.jpg", image, )
    cv2.imshow("Output", image)
    cv2.waitKey(0)


# OpenCV Haar Sınıflandırıcı ile Maske Tanımlama

In [8]:
FaceMaskDetection_Picture_With_OpenCV_HaarCascade(
    "test_images/0072.jpg", 
    "mask_detector.model", 
    r'face_detector\haarcascade_frontalface_default.xml')

> 4 adet yüz bulundu!


# OpenCV DNN Sınıflandırıcı ile Maske Tanımlama

In [18]:
FaceMaskDetection_Picture_With_OpenCV_DNN(
    "test_images/0072.jpg", 
    "mask_detector.model", 
    r'face_detector\deploy.prototxt', 
    r'face_detector\res10_300x300_ssd_iter_140000.caffemodel', 
    0.5)


> DNN yüz tanımlama modeli yükleniyor...
> Yüz maskesi tanımlama modeli yükleniyor...
> Yüzler algılanıyor...
> 200 adet yüz olmaya aday nesne bulundu!
> 4 adet yüz belirlendi!


# MTCNN Sınıflandırıcı ile Maske Tanımlama

In [20]:
FaceMaskDetection_Picture_With_MTCNN(
    "test_images/0072.jpg", 
    "mask_detector.model")

> 5 adet yüz bulundu!
